<a href="https://colab.research.google.com/github/pavankumarbalijepalli/pr-phi2-vs-defog/blob/main/Phi2_Merge_GGUF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets
!pip install -q -U torch=='2.0.0'
!pip install -q -U accelerate=='0.25.0' peft=='0.7.1' bitsandbytes=='0.41.3.post2' trl=='0.7.4'
!pip install -q -U transformers einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━

In [3]:
# Data Exploration Imports
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)

In [4]:
from google.colab import drive

drive.mount('drive')

Mounted at drive


In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
from peft import PeftModel

model_name = "microsoft/phi-2"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16
)

peftmodel = PeftModel.from_pretrained(model, "pavankumarbalijepalli/phi2-nl2sql-lora")
peftmodel = peftmodel.merge_and_unload()

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
peftmodel

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_layernorm): LayerNorm((2560,),

In [8]:
peftmodel.save_pretrained("phi2-nl2sql-lora-merged")
tokenizer.save_pretrained("phi2-nl2sql-lora-merged")

('phi2-nl2sql-lora-merged/tokenizer_config.json',
 'phi2-nl2sql-lora-merged/special_tokens_map.json',
 'phi2-nl2sql-lora-merged/vocab.json',
 'phi2-nl2sql-lora-merged/merges.txt',
 'phi2-nl2sql-lora-merged/added_tokens.json',
 'phi2-nl2sql-lora-merged/tokenizer.json')

In [ ]:
import os

if not os.path.exists('out'):
  os.makedirs('out')

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && make
!python3 -m pip install -r requirements.txt

!python llama.cpp/convert-hf-to-gguf.py phi2-nl2sql-lora-merged --outfile out/phi2-nl2sql-lora-merged-f16.gguf --outtype f16

In [9]:
from zipfile import ZipFile
import os

def get_all_file_paths(directory):
    file_paths = []
    for root, directories, files in os.walk(directory):
        for filename in files:
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)
    return file_paths

def zip_it(directory: str, file_name: str):
    file_paths = get_all_file_paths(directory)
    print('Following files will be zipped:')
    for file_path in file_paths:
        print(file_path)
    with ZipFile(file_name,'w') as zip:
        for file in file_paths:
            zip.write(file)
    print('All files zipped successfully!')

In [17]:
zip_it("phi2-nl2sql-lora-merged", "merged.zip")

from datetime import datetime
import shutil

name = 'merged_' + datetime.now().strftime("%Y_%m_%d_%H_%M_%S") + '.zip'
shutil.move("/content/merged.zip", "/content/drive/MyDrive/phi2_finetune/" + name)

Following files will be zipped:
pavankumarbalijepalli/phi2-nl2sql-lora-merged/tokenizer_config.json
pavankumarbalijepalli/phi2-nl2sql-lora-merged/tokenizer.json
pavankumarbalijepalli/phi2-nl2sql-lora-merged/model-00001-of-00002.safetensors
pavankumarbalijepalli/phi2-nl2sql-lora-merged/added_tokens.json
pavankumarbalijepalli/phi2-nl2sql-lora-merged/model-00002-of-00002.safetensors
pavankumarbalijepalli/phi2-nl2sql-lora-merged/special_tokens_map.json
pavankumarbalijepalli/phi2-nl2sql-lora-merged/generation_config.json
pavankumarbalijepalli/phi2-nl2sql-lora-merged/model.safetensors.index.json
pavankumarbalijepalli/phi2-nl2sql-lora-merged/config.json
pavankumarbalijepalli/phi2-nl2sql-lora-merged/vocab.json
pavankumarbalijepalli/phi2-nl2sql-lora-merged/merges.txt
All files zipped successfully!


'/content/drive/MyDrive/phi2_finetune/merged_2024_02_21_09_19_47.zip'

In [13]:
zip_it("out", "phi2-nl2sql-lora-merged-f16.zip")

from datetime import datetime
import shutil

name = 'merged_gguf_' + datetime.now().strftime("%Y_%m_%d_%H_%M_%S") + '.zip'
shutil.move("/content/phi2-nl2sql-lora-merged-f16.zip", "/content/drive/MyDrive/phi2_finetune/" + name)

'/content/drive/MyDrive/phi2_finetune/merged_gguf_2024_02_21_10_35_00.zip'